### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP на implicit данных

Мягкий дедлайн 13 Октября (пишутся замечания, выставляется оценка, есть возможность исправить до жесткого дедлайна)

Жесткий дедлайн 20 Октября (Итоговая проверка)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp

from lightfm.datasets import fetch_movielens

В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [3]:
ratings = pd.read_csv('ml-1m/ratings.dat', delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [4]:
ratings.head()

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [5]:
movie_info = pd.read_csv('ml-1m/movies.dat', delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python', encoding='latin-1')

In [6]:
movie_info.head()

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


Explicit данные

In [7]:
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [8]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [9]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4
10,1,595,5
11,1,938,4
12,1,2398,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [10]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [11]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

В качестве loss здесь всеми любимый RMSE

In [12]:
model.fit(user_item_t_csr)

  0%|          | 0/100 [00:00<?, ?it/s]

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [13]:
movie_info.head(5)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [14]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                        for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [15]:
get_similars(1, model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '33    Babe (1995)',
 '584    Aladdin (1992)',
 '2315    Babe: Pig in the City (1998)',
 '360    Lion King, The (1994)',
 '1526    Hercules (1997)',
 '1838    Mulan (1998)',
 '2618    Tarzan (1999)']

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [16]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [17]:
get_user_history(4, implicit_ratings)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [18]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_csr)]

In [19]:
get_recommendations(4, model)

['585    Terminator 2: Judgment Day (1991)',
 '2502    Matrix, The (1999)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '1182    Aliens (1986)',
 '3402    Close Encounters of the Third Kind (1977)',
 '2460    Planet of the Apes (1968)',
 '847    Godfather, The (1972)',
 '2880    Dr. No (1962)']

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

In [20]:
from RecSys import build_recsys

### Задание 1. Не используя готовые решения, реализовать SVD разложение используя SGD на explicit данных

In [21]:
from models import SGD_bias

In [24]:
sgd_model = SGD_bias(rank=64, lr=1e-2, lambd=1e-2, eps=1e-3, max_epochs=10)
sgd_model.fit(ratings)

Users: 6040 | Movies: 3706 | Ratings: 1000209


100%|██████████| 1000209/1000209 [02:18<00:00, 7244.04it/s]


Epoch: 0 | MSE=0.9065413843754566


100%|██████████| 1000209/1000209 [02:18<00:00, 7229.25it/s]


Epoch: 1 | MSE=0.8258805495441682


...

Epoch: 7 | MSE=0.6564743291098125


100%|██████████| 1000209/1000209 [02:29<00:00, 6670.28it/s]


Epoch: 8 | MSE=0.6196712560869547


100%|██████████| 1000209/1000209 [02:15<00:00, 7369.55it/s]

Epoch: 9 | MSE=0.5840352762079175


In [26]:
SGD_RECSYS = build_recsys(ratings, sgd_model)

In [27]:
get_similars(1, SGD_RECSYS)

['3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '584    Aladdin (1992)',
 '2012    Little Mermaid, The (1989)',
 '2021    Rescuers, The (1977)',
 '1526    Hercules (1997)',
 '591    Beauty and the Beast (1991)',
 '1838    Mulan (1998)',
 '1664    Mouse Hunt (1997)',
 '1016    Dumbo (1941)']

In [28]:
get_recommendations(4, SGD_RECSYS)

['2169    Seven Beauties (Pasqualino Settebellezze) (1976)',
 '3026    Grapes of Wrath, The (1940)',
 '1872    Hamlet (1948)',
 '1211    Annie Hall (1977)',
 '1276    Room with a View, A (1986)',
 '2953    General, The (1927)',
 '3560    Gold Rush, The (1925)',
 '1088    Streetcar Named Desire, A (1951)',
 '1218    Seventh Seal, The (Sjunde inseglet, Det) (1957)',
 '3132    Five Easy Pieces (1970)']

### Задание 2. Не используя готовые решения, реализовать матричное разложение используя ALS на implicit данных

In [29]:
from models import ALS

In [31]:
als_model = ALS(rank=64, confidence=1e4, lambd=1e-2, eps=1e-3, max_epochs=10)
als_model.fit(user_item_csr)

Users: 6038 | Movies: 3533 | Ratings: 575281


100%|██████████| 3533/3533 [00:11<00:00, 315.25it/s]


Epoch: 1 | MSE=0.0002035355328434781


In [32]:
ALS_RECSYS = build_recsys(ratings, als_model)

In [33]:
get_similars(1, ALS_RECSYS)

['1245    Groundhog Day (1993)',
 '3045    Toy Story 2 (1999)',
 '584    Aladdin (1992)',
 '360    Lion King, The (1994)',
 '148    Apollo 13 (1995)',
 '1250    Back to the Future (1985)',
 '33    Babe (1995)',
 '591    Beauty and the Beast (1991)',
 '1246    Unforgiven (1992)',
 '1120    Monty Python and the Holy Grail (1974)']

In [34]:
get_recommendations(4, ALS_RECSYS)

['1192    Star Wars: Episode VI - Return of the Jedi (1983)',
 '1182    Aliens (1986)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 "1176    One Flew Over the Cuckoo's Nest (1975)",
 '900    Casablanca (1942)',
 '3509    Gladiator (2000)',
 "887    Singin' in the Rain (1952)",
 '912    2001: A Space Odyssey (1968)',
 '1239    Stand by Me (1986)',
 '2502    Matrix, The (1999)']

### Задание 3. Не используя готовые решения, реализовать матричное разложение BPR на implicit данных

In [35]:
from models import BPR

In [38]:
bpr_model = BPR(rank=64, lr=1e-1, lambd=1e-6, eps=1e-3, max_epochs=10)
bpr_model.fit(user_item_csr)

Users: 6038 | Movies: 3533 | Ratings: 575281


100%|██████████| 575281/575281 [01:06<00:00, 8640.59it/s]


Epoch: 1 | Loss=-2.237791576830553


100%|██████████| 575281/575281 [01:03<00:00, 9126.91it/s]


Epoch: 2 | Loss=-2.98830091027228


...

Epoch: 8 | Loss=-5.42128347517683


100%|██████████| 575281/575281 [01:04<00:00, 8971.23it/s]


Epoch: 9 | Loss=-5.744525954345731


100%|██████████| 575281/575281 [01:03<00:00, 9001.63it/s]

Epoch: 10 | Loss=-6.068193024845436


In [39]:
BPR_RECSYS = build_recsys(ratings, bpr_model)

In [40]:
get_similars(1, BPR_RECSYS)

['1495    Sprung (1997)',
 '645    Superweib, Das (1996)',
 '3045    Toy Story 2 (1999)',
 '810    Crude Oasis, The (1995)',
 '3050    Bay of Blood (Reazione a catena) (1971)',
 '2145    Number Seventeen (1932)',
 '1102    Tashunga (1995)',
 '587    Tough and Deadly (1995)',
 "597    Wooden Man's Bride, The (Wu Kui) (1994)",
 '3304    Buck and the Preacher (1972)']

In [41]:
get_recommendations(4, BPR_RECSYS)

['1178    Star Wars: Episode V - The Empire Strikes Back...',
 '1182    Aliens (1986)',
 '585    Terminator 2: Judgment Day (1991)',
 '1179    Princess Bride, The (1987)',
 '847    Godfather, The (1972)',
 '2502    Matrix, The (1999)',
 '1192    Star Wars: Episode VI - Return of the Jedi (1983)',
 '108    Braveheart (1995)',
 '589    Silence of the Lambs, The (1991)',
 '453    Fugitive, The (1993)']

### Задание 4. Не используя готовые решения, реализовать матричное разложение WARP на implicit данных

In [42]:
from models import WARP

In [44]:
warp_model = WARP(rank=64, lr=1e-2, lambd=1e-6, eps=1e-3, max_epochs=10)
warp_model.fit(user_item_csr)

Users: 6038 | Movies: 3533 | Ratings: 575281


100%|██████████| 575281/575281 [02:47<00:00, 3425.26it/s]


Epoch: 1 | Loss=1.1536096018103819


100%|██████████| 575281/575281 [03:20<00:00, 2865.11it/s]


Epoch: 2 | Loss=1.063036333837461


...

Epoch: 8 | Loss=0.585982783009212


100%|██████████| 575281/575281 [04:23<00:00, 2183.14it/s]


Epoch: 9 | Loss=0.5462712569432828


100%|██████████| 575281/575281 [04:27<00:00, 2147.16it/s]


Epoch: 10 | Loss=0.5171068724684411


In [45]:
WARP_RECSYS = build_recsys(ratings, warp_model)

In [46]:
get_similars(1, WARP_RECSYS)

['3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '33    Babe (1995)',
 '1495    Sprung (1997)',
 '1245    Groundhog Day (1993)',
 '584    Aladdin (1992)',
 '3054    Spring Fever USA (a.k.a. Lauderdale) (1989)',
 '2465    Avalanche (1978)',
 "597    Wooden Man's Bride, The (Wu Kui) (1994)",
 '645    Superweib, Das (1996)']

In [47]:
get_recommendations(4, WARP_RECSYS)

['847    Godfather, The (1972)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '1203    Godfather: Part II, The (1974)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '1192    Star Wars: Episode VI - Return of the Jedi (1983)',
 '1182    Aliens (1986)',
 '585    Terminator 2: Judgment Day (1991)',
 '589    Silence of the Lambs, The (1991)',
 '537    Blade Runner (1982)',
 "1176    One Flew Over the Cuckoo's Nest (1975)"]